### Carregamento das libs

In [1]:
from src.libs.lib import *
import pandas as pd
import matplotlib.pyplot as plt
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

Hash numérica de 5 dígitos: 94799


# all_contracts

In [2]:
all_contracts = pd.read_csv("src/data/all_contracts.csv")
all_contracts = all_contracts.loc[:, ~all_contracts.columns.str.contains('^Unnamed')]

all_contracts.head()

,created_at,freight_id,driver_id,tipo,nivel_px
0,2023-04-28 15:50:15.000,22059,64674,Ajudante,NaN
1,2023-05-10 14:20:02.000,23068,29889,Diária,Ouro
2,2022-11-28 16:22:58.000,11319,26879,Diária,Diamante
3,2022-11-28 16:37:00.000,11320,12224,Diária,Ouro
4,2022-09-08 17:05:24.000,8436,18608,Diária,Ouro


# df_original

In [3]:
df_original_inicial = pd.read_csv("src/data/contratos_adiados.csv")
df_original_inicial = df_original_inicial.loc[:, ~df_original_inicial.columns.str.contains('^Unnamed')]

df_original_inicial.head()

,created_at,id_adiamento,dt_alteracao,freight_id,old_value_start,new_value_start,horas_adicionadas,tipo,driver_id,concluido,qt_candidaturas_antes_adiamento,qt_selecionados_antes_adiamento
0,2025-03-22 20:09:17.000,3040413,2025-03-24 16:36:37.000,151948,2025-03-25 08:00:00.000,2025-03-26 08:00:00.000,24.0,Diária,44446,Não,2,0
1,2025-03-22 20:09:17.000,3047514,2025-03-25 16:59:32.000,151948,2025-03-26 08:00:00.000,2025-03-27 08:00:00.000,24.0,Diária,44446,Não,3,0
2,2025-03-22 20:09:17.000,3077042,2025-03-31 07:58:29.000,151948,2025-03-27 08:00:00.000,2025-04-02 08:00:00.000,144.0,Diária,44446,Não,3,0
3,2025-03-22 20:09:17.000,3101483,2025-04-02 14:11:01.000,151948,2025-04-02 08:00:00.000,2025-04-04 08:00:00.000,48.0,Diária,44446,Não,5,1
4,2025-03-22 20:09:17.000,3113234,2025-04-04 11:15:58.000,151948,2025-04-04 08:00:00.000,2025-04-07 08:00:00.000,72.0,Diária,44446,Não,5,1


In [4]:
import pandas as pd
import re

# eliminando contratos com datas inválidas
cols_to_check = ['old_value_start', 'new_value_start', 'created_at']

date_pattern = re.compile(r"^(\d{4})-\d{2}-\d{2}")

def year_out_of_bounds(val):
    if isinstance(val, str):
        match = date_pattern.match(val)
        if match:
            year = int(match.group(1))
            return year < 1677 or year > 2262
    return False

freights_com_erro = set()

for col in cols_to_check:
    mask = df_original_inicial[col].apply(year_out_of_bounds)
    freights_com_erro.update(df_original_inicial.loc[mask, 'freight_id'].tolist())

df_invalidos = df_original_inicial[df_original_inicial['freight_id'].isin(freights_com_erro)]


df_original_inicial = df_original_inicial.merge(
    df_invalidos[['freight_id', 'old_value_start', 'new_value_start']], 
    on=['freight_id', 'old_value_start', 'new_value_start'], 
    how='left', 
    indicator=True
)

df_original_inicial = df_original_inicial[df_original_inicial['_merge'] == 'left_only'].drop(columns=['_merge'])
df_original_inicial[cols_to_check] = df_original_inicial[cols_to_check].apply(pd.to_datetime)

df_original_inicial.head()

,created_at,id_adiamento,dt_alteracao,freight_id,old_value_start,new_value_start,horas_adicionadas,tipo,driver_id,concluido,qt_candidaturas_antes_adiamento,qt_selecionados_antes_adiamento
0,2025-03-22 20:09:17,3040413,2025-03-24 16:36:37.000,151948,2025-03-25 08:00:00,2025-03-26 08:00:00,24.0,Diária,44446,Não,2,0
1,2025-03-22 20:09:17,3047514,2025-03-25 16:59:32.000,151948,2025-03-26 08:00:00,2025-03-27 08:00:00,24.0,Diária,44446,Não,3,0
2,2025-03-22 20:09:17,3077042,2025-03-31 07:58:29.000,151948,2025-03-27 08:00:00,2025-04-02 08:00:00,144.0,Diária,44446,Não,3,0
3,2025-03-22 20:09:17,3101483,2025-04-02 14:11:01.000,151948,2025-04-02 08:00:00,2025-04-04 08:00:00,48.0,Diária,44446,Não,5,1
4,2025-03-22 20:09:17,3113234,2025-04-04 11:15:58.000,151948,2025-04-04 08:00:00,2025-04-07 08:00:00,72.0,Diária,44446,Não,5,1


In [5]:
df_original_inicial.qt_candidaturas_antes_adiamento.describe()

count    5.000000
mean     3.600000
std      1.341641
min      2.000000
25%      3.000000
50%      3.000000
75%      5.000000
max      5.000000
Name: qt_candidaturas_antes_adiamento, dtype: float64

In [6]:
df_original_inicial.qt_selecionados_antes_adiamento.describe()

count    5.000000
mean     0.400000
std      0.547723
min      0.000000
25%      0.000000
50%      0.000000
75%      1.000000
max      1.000000
Name: qt_selecionados_antes_adiamento, dtype: float64

In [7]:
df_original_inicial['ano_mes'] = df_original_inicial['created_at'].dt.to_period('M').dt.to_timestamp()

df_original_inicial = df_original_inicial.drop(columns=['created_at', 'id_adiamento'])

df_original_inicial = df_original_inicial.rename(columns={'horas_adicionadas': 'horas_adiadas'})
df_original_inicial['horas_adiadas'] = np.ceil(df_original_inicial['horas_adiadas'] / 24) * 24



# ajustando o mesmo período para os contratos já analisados em remocao_dias
df_original_inicial = df_original_inicial.loc[(df_original_inicial.ano_mes > '2024-10-01') & (df_original_inicial.ano_mes <'2025-04-01') ]

# Criar colunas de interesse
df_original_inicial['houve_candidatura'] = df_original_inicial['qt_candidaturas_antes_adiamento'].apply(
    lambda x: 'Sim' if x != 0 else 'Não')

df_original_inicial['houve_selecionado'] = df_original_inicial['qt_selecionados_antes_adiamento'].apply(
    lambda x: 'Sim' if x != 0 else 'Não')


df_original_inicial.head()

,dt_alteracao,freight_id,old_value_start,new_value_start,horas_adiadas,tipo,driver_id,concluido,qt_candidaturas_antes_adiamento,qt_selecionados_antes_adiamento,ano_mes,houve_candidatura,houve_selecionado
0,2025-03-24 16:36:37.000,151948,2025-03-25 08:00:00,2025-03-26 08:00:00,24.0,Diária,44446,Não,2,0,2025-03-01,Sim,Não
1,2025-03-25 16:59:32.000,151948,2025-03-26 08:00:00,2025-03-27 08:00:00,24.0,Diária,44446,Não,3,0,2025-03-01,Sim,Não
2,2025-03-31 07:58:29.000,151948,2025-03-27 08:00:00,2025-04-02 08:00:00,144.0,Diária,44446,Não,3,0,2025-03-01,Sim,Não
3,2025-04-02 14:11:01.000,151948,2025-04-02 08:00:00,2025-04-04 08:00:00,48.0,Diária,44446,Não,5,1,2025-03-01,Sim,Sim
4,2025-04-04 11:15:58.000,151948,2025-04-04 08:00:00,2025-04-07 08:00:00,72.0,Diária,44446,Não,5,1,2025-03-01,Sim,Sim


# df_original

In [8]:
df_original = df_original_inicial[['ano_mes','dt_alteracao', 'tipo', 'freight_id', 
       'horas_adiadas',  'concluido', 'houve_candidatura', 'houve_selecionado']]

df_original

,ano_mes,dt_alteracao,tipo,freight_id,horas_adiadas,concluido,houve_candidatura,houve_selecionado
0,2025-03-01,2025-03-24 16:36:37.000,Diária,151948,24.0,Não,Sim,Não
1,2025-03-01,2025-03-25 16:59:32.000,Diária,151948,24.0,Não,Sim,Não
2,2025-03-01,2025-03-31 07:58:29.000,Diária,151948,144.0,Não,Sim,Não
3,2025-03-01,2025-04-02 14:11:01.000,Diária,151948,48.0,Não,Sim,Sim
4,2025-03-01,2025-04-04 11:15:58.000,Diária,151948,72.0,Não,Sim,Sim


In [9]:
df_original.horas_adiadas.describe()

count      5.000000
mean      62.400000
std       49.767459
min       24.000000
25%       24.000000
50%       48.000000
75%       72.000000
max      144.000000
Name: horas_adiadas, dtype: float64

In [10]:
df_original.horas_adiadas.unique()

array([ 24., 144.,  48.,  72.])

In [11]:
# Calcular limites usando IQR 
q1 = df_original['horas_adiadas'].quantile(0.25)
q3 = df_original['horas_adiadas'].quantile(0.75)
iqr = q3 - q1

limite_inferior = q1 - 2 * iqr
limite_superior = q3 + 2 * iqr

# Filtrar dados sem outliers
df_sem_outliers = df_original[
    (df_original['horas_adiadas'] >= limite_inferior) &
    (df_original['horas_adiadas'] <= limite_superior)
]
df_sem_outliers.horas_adiadas.value_counts()

horas_adiadas
24.0     2
144.0    1
48.0     1
72.0     1
Name: count, dtype: int64

In [12]:
df_sem_outliers.tipo.value_counts()

tipo
Diária    5
Name: count, dtype: int64

In [13]:
df_sem_outliers.to_csv("src/data/df_original_dias_adiados.csv")

# df_original_contratos_adiados

In [14]:
df_original_contratos_adiados = df_original_inicial[['freight_id', 'tipo','created_at', 'horas_adiadas', 'concluido', 'houve_candidatura', 'houve_selecionado']]

df_original_contratos_adiados

KeyError: "['created_at'] not in index"

In [ ]:
df_original_contratos_adiados.loc[df_original_contratos_adiados.freight_id == 151948]

In [ ]:
df_original_contratos_adiados = (
    df_original_contratos_adiados
    .groupby(['freight_id', 'tipo'], as_index=False)
    .agg(
        horas_adiadas_total=('horas_adiadas', 'sum'),
        concluido=('concluido', lambda x: 'Sim' if (x == 'Sim').any() else 'Não'),
        houve_candidatura=('houve_candidatura', lambda x: 'Sim' if (x == 'Sim').any() else 'Não'),
        houve_selecionado=('houve_selecionado', lambda x: 'Sim' if (x == 'Sim').any() else 'Não')
    )
)

In [ ]:
df_original_contratos_adiados.loc[df_original_contratos_adiados.freight_id == 151948]

In [ ]:
df_original_contratos_adiados = (
    df_original_contratos_adiados
    .groupby(['freight_id', 'tipo'], as_index=False)
    .agg(
        horas_adiadas_total=('horas_adiadas', 'sum'),
        concluido=('concluido', lambda x: 'Sim' if (x == 'Sim').any() else 'Não')
    )
)

df_original_contratos_adiados

In [ ]:
# Calcular limites usando IQR 
q1 = df_original_contratos_adiados['horas_adiadas_total'].quantile(0.25)
q3 = df_original_contratos_adiados['horas_adiadas_total'].quantile(0.75)
iqr = q3 - q1

limite_inferior = q1 - 3 * iqr
limite_superior = q3 + 3 * iqr

# Filtrar dados sem outliers
df_original_contratos_adiados_sem_outlier = df_original_contratos_adiados[
    (df_original_contratos_adiados['horas_adiadas_total'] >= limite_inferior) &
    (df_original_contratos_adiados['horas_adiadas_total'] <= limite_superior)
]
df_original_contratos_adiados_sem_outlier.horas_adiadas_total.value_counts()

In [ ]:
df_original_contratos_adiados_sem_outlier.loc[df_original_contratos_adiados.freight_id == 151948]

In [ ]:
df_original_contratos_adiados_sem_outlier.to_csv("src/data/df_original_contratos_adiados.csv")